In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
import os
import torch
import pandas as pd
import numpy as np
import sys
sys.path.append("../src")
from utils import simple_mean_average_precision, StrawberryDataset, collate_fn, get_transform, get_model, load_data, get_optimizer

from torch.utils.data import DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import gc

train_dir = "../strawberry/train/"
val_dir = "../strawberry/val/"
test_dir = "../strawberry/test/"

train_images, df_train_labels = load_data(train_dir)
test_images, df_test_labels = load_data(val_dir)
val_images, df_val_labels = load_data(test_dir)

100%|██████████| 743/743 [00:00<00:00, 4083.30it/s]


In [3]:
batch_size = 2

gc.collect()
torch.cuda.empty_cache()

train_data = StrawberryDataset(train_images, df_train_labels, transforms=get_transform)
val_data = StrawberryDataset(test_images, df_test_labels, transforms=get_transform)
test_data = StrawberryDataset(val_images, df_val_labels, transforms=get_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=collate_fn)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

num_classes = 7
model = get_model("baseline", num_classes)
model.to(device)

gradient_accumulation_steps = 4
params = [p for p in model.parameters() if p.requires_grad] 
model, optimizer = get_optimizer("SGD", model, lr=0.0001)
scaler = GradScaler()
num_epochs = 50
history = pd.DataFrame(columns=["Epoch", "Train Loss", "Val Loss", "Train mAP", "Val mAP"])
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    train_mAP = 0
    total_train_samples = 0
    step = 0
    
    torch.cuda.synchronize()
    gc.collect()
    torch.cuda.empty_cache()
    
    for images, targets in tqdm(train_loader, desc=f"Training epoch {epoch + 1}/{num_epochs}"):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with autocast():
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        scaler.scale(losses / gradient_accumulation_steps).backward()
        running_loss += losses.cpu().detach().item()
        total_train_samples += len(targets)
        if (step + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        step += 1

    model.eval()
    val_loss = 0
    val_mAP = []
    total_val_samples = 0
    predictions = []

    mAP_sums = {}
    with torch.no_grad():
        for images, targets in tqdm(val_loader, desc=f"Validating epoch {epoch + 1}/{num_epochs}"):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            with autocast():
                predictions = model(images)
            mAP_dict = simple_mean_average_precision(targets, predictions)
            for key, value in mAP_dict.items():
                if key in mAP_sums:
                    mAP_sums[key] += value.cpu().detach().numpy()
                else:
                    mAP_sums[key] = value.numpy()
            total_val_samples += len(targets)

    # Calculate the average of each mAP metric
    mAP_averages = {key: value / total_val_samples for key, value in mAP_sums.items()}
    val_mAP.append(mAP_averages)

    train_loss = running_loss / (step + 1)
    print(f"Epoch: {epoch + 1}, Train Loss: {train_loss}, Val mAP: {mAP_averages}")
    history = history.append({
        "Epoch": epoch + 1,
        "Train Loss": train_loss,
        "Val mAP": val_mAP},
        ignore_index=True)

history.to_csv("../models/training_history_base.csv", index=False)
torch.save(model.state_dict(), "../models/mask_rcnn_strawberry_base.pth")

# Evaluate the model on the test set
model.eval()
test_mAP = []
total_test_samples = 0
predictions_test = []
mAP_test_sums = {}
with torch.no_grad():
    for images, targets in test_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        with autocast():
            predictions += model(images)  # Get predictions
            mAP_dict = simple_mean_average_precision(targets, predictions)

            # Update the running sum of each mAP metric
            for key, value in mAP_dict.items():
                if key in mAP_test_sums:
                    mAP_test_sums[key] += value.cpu().detach().numpy()
                else:
                    mAP_test_sums[key] = value.cpu().detach().numpy()
            total_test_samples += len(targets)

    # Calculate the average of each mAP metric
    mAP_test_averages = {key: value / total_val_samples for key, value in mAP_test_sums.items()}
    print(f"Test mAP: {mAP_test_averages}")

print("Finished evaluating the model on the test set.")

cuda


c:\Users\bruno\.conda\envs\my_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bruno\.conda\envs\my_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Validating epoch 1/50: 100%|██████████| 154/154 [00:33<00:00,  4.59it/s]
C:\Users\bruno\AppData\Local\Temp\ipykernel_11200\2608262362.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history = history.append({


Epoch: 1, Train Loss: 1.2666958819243535, Val mAP: {'map': 0.006008303514909278, 'map_50': 0.02286094330032796, 'map_75': 0.0006617578504916511, 'map_small': -0.501628664495114, 'map_medium': -0.2671009771986971, 'map_large': -0.02577199842720156, 'mar_1': 0.003279432023387002, 'mar_10': 0.014806253513994744, 'mar_100': 0.024924262727122354, 'mar_small': -0.501628664495114, 'mar_medium': -0.2671009771986971, 'mar_large': -0.003991906340036796, 'map_per_class': -0.501628664495114, 'mar_100_per_class': -0.501628664495114}


Validating epoch 2/50: 100%|██████████| 154/154 [00:51<00:00,  2.98it/s]
C:\Users\bruno\AppData\Local\Temp\ipykernel_11200\2608262362.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history = history.append({


Epoch: 2, Train Loss: 1.0364733126897048, Val mAP: {'map': 0.011450322520849371, 'map_50': 0.044022662631851846, 'map_75': 0.001788702772183993, 'map_small': -0.501628664495114, 'map_medium': -0.2671009771986971, 'map_large': -0.019618717777612545, 'mar_1': 0.004943345191036063, 'mar_10': 0.02789733774887234, 'mar_100': 0.05004129502982969, 'mar_small': -0.501628664495114, 'mar_medium': -0.2671009771986971, 'mar_large': 0.02502560227235676, 'map_per_class': -0.501628664495114, 'mar_100_per_class': -0.501628664495114}


...

Epoch: 48, Train Loss: 0.499014983425743, Val mAP: {'map': 0.1880403326078036, 'map_50': 0.3344355033352631, 'map_75': 0.1862844286990088, 'map_small': -0.501628664495114, 'map_medium': -0.21554160351085352, 'map_large': 0.16158186573935643, 'mar_1': 0.11281276292831968, 'mar_10': 0.21200792641903757, 'mar_100': 0.21834606456445949, 'mar_small': -0.501628664495114, 'mar_medium': -0.20075230800367722, 'mar_large': 0.18811257576709461, 'map_per_class': -0.501628664495114, 'mar_100_per_class': -0.501628664495114}


Validating epoch 49/50: 100%|██████████| 154/154 [00:32<00:00,  4.76it/s]
C:\Users\bruno\AppData\Local\Temp\ipykernel_11200\2608262362.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history = history.append({


Epoch: 49, Train Loss: 0.48732685250391766, Val mAP: {'map': 0.18353276454664597, 'map_50': 0.3292213961821814, 'map_75': 0.17082266543509517, 'map_small': -0.501628664495114, 'map_medium': -0.21988618335040463, 'map_large': 0.15976329890446864, 'mar_1': 0.10999381348053873, 'mar_10': 0.20816026370766108, 'mar_100': 0.21126601129090747, 'mar_small': -0.501628664495114, 'mar_medium': -0.20504899910296212, 'mar_large': 0.1827015084630115, 'map_per_class': -0.501628664495114, 'mar_100_per_class': -0.501628664495114}


Validating epoch 50/50: 100%|██████████| 154/154 [00:32<00:00,  4.81it/s]
C:\Users\bruno\AppData\Local\Temp\ipykernel_11200\2608262362.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history = history.append({


Epoch: 50, Train Loss: 0.4824119851088726, Val mAP: {'map': 0.1930286519302219, 'map_50': 0.33315821346320235, 'map_75': 0.18159366818903325, 'map_small': -0.501628664495114, 'map_medium': -0.21550087198760687, 'map_large': 0.16859055808002088, 'mar_1': 0.11839361532503308, 'mar_10': 0.2173691541441877, 'mar_100': 0.22145140209881412, 'mar_small': -0.501628664495114, 'mar_medium': -0.20419020761496082, 'mar_large': 0.19376372048443224, 'map_per_class': -0.501628664495114, 'mar_100_per_class': -0.501628664495114}
Test mAP: {'map': 0.023405547250753892, 'map_50': 0.04734071613523005, 'map_75': 0.02517173344615228, 'map_small': -1.208469055374593, 'map_medium': -0.7877252932868485, 'map_large': -0.1289442851411403, 'mar_1': 0.022401115405054746, 'mar_10': 0.024650483643969805, 'mar_100': 0.02770818095253811, 'mar_small': -1.208469055374593, 'mar_medium': -0.7867535022648615, 'mar_large': -0.12511296225681368, 'map_per_class': -1.211726384364821, 'mar_100_per_class': -1.211726384364821}
Fi